In [1]:
import torch
from datasets import load_from_disk
from transformers import (AutoModelForCausalLM,
                          AutoTokenizer,
                          BitsAndBytesConfig,
                          TrainingArguments,
                          LlamaTokenizer)
from peft import (LoraConfig,
                  get_peft_model,
                  prepare_model_for_kbit_training)
import bitsandbytes as bnb
from trl import SFTTrainer


bin C:\ProgramData\Miniconda\envs\qlora\lib\site-packages\bitsandbytes\libbitsandbytes_cuda118.dll


In [2]:
# Load dataset
dataset = load_from_disk('data/finetuning_llama')['train']


In [3]:
# Specify model path
model_path = "llms/Llama-2-13b-chat-hf"

# Load model
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    load_in_4bit=True,
    device_map="auto",
    quantization_config=BitsAndBytesConfig(
        load_in_4bit=True,
        llm_int8_threshold=6.0,
        llm_int8_has_fp16_weight=False,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16
    ),
    trust_remote_code=True
)

model.config.use_cache = False


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    model_path,
    padding_side="right",
    use_fast=False,
    trust_remote_code=True
)

tokenizer.pad_token = tokenizer.eos_token


In [5]:
# PEFT preparation
model = prepare_model_for_kbit_training(model, use_gradient_checkpointing=True)


In [6]:
# PEFT LoRA configuration
# github.com/artidoro/qlora/blob/main/qlora.py
def find_linear(model):
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, bnb.nn.Linear4bit):
            names = name.split('.')
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])

    if 'lm_head' in lora_module_names:
        lora_module_names.remove('lm_head')
    return list(lora_module_names)


modules = find_linear(model)
peft_config = LoraConfig(
    r=128,
    lora_alpha=16,
    lora_dropout=0.05,
    target_modules=modules,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, peft_config)
for name, module in model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)


In [7]:
# Define training arguments
# github.com/artidoro/qlora/blob/main/qlora.py
training_arguments = TrainingArguments(
    per_device_train_batch_size=1,
    gradient_accumulation_steps=16,
    optim="paged_adamw_32bit",
    save_steps=20,
    save_strategy='steps',
    logging_steps=10,
    learning_rate=2e-4,
    fp16=True,
    max_grad_norm=0.3,
    num_train_epochs=2,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    output_dir="checkpoints/llama_128",
)


In [8]:
# Setup STFTrainer
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    dataset_text_field="conversation",
    max_seq_length=2048,
    tokenizer=tokenizer,
    peft_config=peft_config,
    args=training_arguments
)


In [9]:
trainer.train()


wandb: Currently logged in as: sean-sun-22. Use `wandb login --relogin` to force relogin


Step,Training Loss
10,1.593500
20,1.265000
30,1.233000
40,1.234200
50,1.175700
60,1.193200
70,1.180200
80,1.154900
90,1.122400
100,1.111500


TrainOutput(global_step=110, training_loss=1.2219526637684215, metrics={'train_runtime': 1876.7845, 'train_samples_per_second': 0.948, 'train_steps_per_second': 0.059, 'total_flos': 4.06198408691712e+16, 'train_loss': 1.2219526637684215, 'epoch': 1.98})

In [10]:
model.save_pretrained("lora/llama_128")

In [11]:
# load lora
# lora_config = LoraConfig.from_pretrained('lora/llama')
# model = get_peft_model(model, lora_config)
# model.config.use_cache = True